#### 개요
학습용으로 제공받은 데이터를 전처리에 사용할 수 있도록 공사비 단위로 하나의 레코드로 취합하는 역활을 수행하며, 학습 대상이 아닌 불필요한 컬럼을 제거함
* STEP01: 각 데이터 셋에서 학습대상 레코드와 컬럼 추출
* STEP02: 공사비별 전주/전선/인입선 갯 수 계산
* STEP03: 컬럼 추가
* STEP99: 전주순서 계산(이번 생에는 순서 계산은 안 할 예정임)

In [1]:
import os
import sys
import pandas as pd

import aidd.sys.config as cfg
from aidd.batch.data_manager import get_provide_data, read_data
from aidd.online.data_manager import get_online_sample

In [2]:
pdata = get_provide_data()

[3b945f25ddb8][2024-03-13 15:56:19.736632] 제공받은 데이터 불러오기 시작
[3b945f25ddb8][2024-03-13 15:56:48.346202]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:28.609502
[3b945f25ddb8][2024-03-13 15:57:13.028312]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:24.681971
[3b945f25ddb8][2024-03-13 15:57:44.790351]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:31.761891
[3b945f25ddb8][2024-03-13 15:57:57.811890]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:13.021239
[3b945f25ddb8][2024-03-13 15:57:57.811973] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:38.075344


#### STEP01: 학습대상 추출

In [3]:
# 모델에 사용할 데이터 딕셔너리
mdict = {}

# 공사비 데이터 학습대상 레코드/컬럼 추출
key = 'CONS'
df = pdata[key]
# (전주/전선 수를 제외한) 공사비 데이터 부분에서 학습 대상 레코드 조건
# * 접수종류명(ACC_TYPE_NAME), 계약전력(CONT_CAP),
# * 공사형태코드(CONS_TYPE_CD), 총공사비(TOTAL_CONS_COST)
modeling_records = \
    (df.ACC_TYPE_NAME == cfg.COND_ACC_TYPE_NAME) & \
    (df.CONT_CAP < cfg.COND_MAX_CONT_CAP) & \
    (df.CONS_TYPE_CD == cfg.COND_CONS_TYPE_CD) & \
    (df.TOTAL_CONS_COST < cfg.COND_MAX_TOTAL_CONS_COST)
df = df[modeling_records].reset_index(drop=True)
cons_df = df[cfg.MODELING_COLS[key]]
mdict[key] = cons_df

In [4]:
# 전주/전선/인입선 데이터 학습대상 레코드/컬럼 추출
# * 레크도는 공사비에 있는 공사번호를 대상으로 함
for key in cfg.DATA_TYPE[1:]:
    df = pdata[key]
    df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
    mdict[key] = df[cfg.MODELING_COLS[key]]

In [5]:
# (참조용)데이터 셋 별 공사대상 레코드/컬럼 크기 확인
for key in cfg.DATA_TYPE:
    print(f'{key}: {mdict[key].shape}')

CONS: (15335, 8)
POLE: (27896, 6)
LINE: (30348, 11)
SL: (17034, 6)


#### STEP02: 공사비별 전주/전선/인입선 갯 수 계산
* 전주/전선 갯 수가 10개 이상인 레코드 제거

In [6]:
# 여기서부터 모델 학습에 사용할 데이터프레임 생성(mdf)
mdf = cons_df
for key in cfg.DATA_TYPE[1:]:
    df = mdict[key]
    count_cons_ids = df.CONS_ID.value_counts()
    col_name = f'{key}_CNT'
    mdf = pd.merge(
        mdf, count_cons_ids.rename(col_name),
        left_on='CONS_ID', right_on=count_cons_ids.index, how='left'
    )
    # 해당 공사번호에 없는 설비는 NaN처리 되며, 이 값을 0으로 변경
    mdf[col_name].fillna(0, inplace=True)

print(f'MDF: {mdf.shape}') 

MDF: (15335, 11)


In [7]:
# 모델 학습에 사용할 레코드 추출
# * 전주/전선 갯 수가 10개 이상인 경우 
modeling_records = \
    (mdf.POLE_CNT >= cfg.COND_MIN_POLE_COUNT) & \
    (mdf.POLE_CNT <= cfg.COND_MAX_POLE_COUNT) & \
    (mdf.LINE_CNT >= cfg.COND_MIN_LINE_COUNT) & \
    (mdf.LINE_CNT <= cfg.COND_MAX_LINE_COUNT)
mdf = mdf[modeling_records]

print(f'MDF: {mdf.shape}')

MDF: (14729, 11)


In [8]:
mdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14729 entries, 0 to 15334
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CONS_ID          14729 non-null  object        
 1   TOTAL_CONS_COST  14729 non-null  int64         
 2   CONS_TYPE_CD     14729 non-null  int64         
 3   LAST_MOD_DATE    14729 non-null  datetime64[ns]
 4   LAST_MOD_EID     14729 non-null  object        
 5   OFFICE_NAME      14729 non-null  object        
 6   CONT_CAP         14729 non-null  int64         
 7   ACC_TYPE_NAME    14729 non-null  object        
 8   POLE_CNT         14729 non-null  float64       
 9   LINE_CNT         14729 non-null  float64       
 10  SL_CNT           14729 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(4)
memory usage: 1.3+ MB
